<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Academics/AI%20Project/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as ureq
ureq.urlretrieve('https://raw.githubusercontent.com/TXH2020/MainRepo/main/Academics/AI%20Project/intents.json','intents.json')

('intents.json', <http.client.HTTPMessage at 0x7f9a9f1307c0>)

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
# things we need for Tensorflow
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype='object')

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
# reset underlying graph data
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.layers.Dense(8,input_shape=(len(train_x[0]),),kernel_initializer='normal'),
    keras.layers.Dense(8,kernel_initializer='normal'),
    keras.layers.Dense(len(train_y[0]),activation='softmax',kernel_initializer='normal')
])
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y), epochs=1000, batch_size=8, verbose=1)

Epoch 1/1000
10/10 [==============================] - 2s 12ms/step - loss: 0.0151 - accuracy: 0.0253
Epoch 2/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0151 - accuracy: 0.1139
Epoch 3/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.0151 - accuracy: 0.1266
Epoch 4/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.0151 - accuracy: 0.1139
Epoch 5/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.0151 - accuracy: 0.1013
Epoch 6/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0151 - accuracy: 0.1013
Epoch 7/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0151 - accuracy: 0.1266
Epoch 8/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0151 - accuracy: 0.1519
Epoch 9/1000
10/10 [==============================] - 0s 11ms/step - loss: 0.0151 - accuracy: 0.1646
Epoch 10/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0151 - accuracy: 0.1

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
['Accounts', 'Admin', 'Chemistry', 'Electrical', 'Electronics', 'FAQ', 'Hostel', 'MCA', 'Mathematics', 'Mechanical', 'Physics', 'Rajyotsava', 'apex', 'canteen', 'courses', 'cse', 'exam', 'goodbye', 'greeting', 'holiday', 'library', 'opentoday', 'parking', 'payments', 'principal', 'seats', 'seminar', 'sports', 'teaching', 'thanks', 'tomorrow']


In [9]:
print(model.predict(np.array([p])))

1/1 [==============================] - 0s 78ms/step
[[9.63388310e-12 2.17969614e-15 1.68900249e-16 6.44781649e-16
  1.51183120e-15 1.48676635e-08 1.88669965e-05 2.21775154e-09
  1.13608423e-19 1.04080931e-18 2.98899157e-14 2.66509753e-12
  5.57306690e-09 7.07132323e-03 1.56096289e-10 1.85131272e-12
  4.22748045e-14 1.53653044e-03 1.37253519e-05 7.92932212e-07
  3.77915072e-04 9.88612831e-01 2.09049634e-18 2.35478114e-03
  2.28699646e-11 4.53226900e-10 7.38550321e-14 1.59081526e-10
  1.17185539e-10 4.07996997e-10 1.31395445e-05]]


In [10]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict(np.array([bow(sentence, words)]))[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [11]:
model.save('model.h5')

In [12]:
import tensorflow as tf
m2=tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(m2)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [13]:
import pickle
with open('intents.pkl','wb') as f:
  pickle.dump(intents,f)
with open('classes.pkl','wb') as f:
  pickle.dump(classes,f)
with open('words.pkl','wb') as f:
  pickle.dump(words,f)